In [1]:
import numpy as np
import networkx as nx
from collections import Counter
import matplotlib.pyplot as plt

def merge(lists):
    out = []
    for l in lists:
        out += l
    return out

# Read Data

In [2]:
def read_data(name):
    railway = {}
    with open('data/train_'+name+'.txt') as f:
        for line in f.readlines():
            rail_name, station = line.split(':')
            railway[rail_name] = station[:-1].split(',')
    return railway

In [3]:
railway_type = ['g','d','k','t','c','z','n']
railway = {r_type:read_data(r_type) for r_type in railway_type}

# Generate Graph

We generate following graphs for different use cases:

1. Stratified Graph
2. Directed Whole Graph
3. Undirected Whole Graph

In [4]:
def Dir2Undir(graph):
    graph_un = graph.to_undirected()
    for edge in graph.edges:
        weight1 = graph[edge[0]][edge[1]]['weight']
        try:
            weight2 = graph[edge[1]][edge[0]]['weight']
        except:
            weight2 = 0
        graph_un[edge[0]][edge[1]]['weight'] = weight1 + weight2
    return graph_un

In [5]:
def GenGraph(rail_dict,directed=True):
    nodes = list(set(merge(rail_dict.values())))
    
    lines = list(rail_dict.values())
    line_connect = [[line[i]+'-'+line[i+1] for i in range(len(line)-1)]\
                    for line in lines]
    line_connect = merge(line_connect)
    line_connect = dict(Counter(line_connect))
    
    graph = nx.DiGraph()
    for node in nodes:
        graph.add_node(node,name = node)
    
    for key, value in line_connect.items():
        depart, dest = key.split('-')
        if depart!=dest:
            graph.add_edge(depart, dest, weight = value)
    
    if not directed:
        graph = Dir2Undir(graph)
    
    return graph

For the directed graph, the weight of edge `a->b` equals to the number of trains passing this two stations continously. And for the undirected graph, the weight of edge `(a,b)` equals to the sum of `a->b` and `b->a` of that in directed graph.

We can build a directed graph by:

```
graph = GenGraph(rail_dict)
```

and undirected graph by:

```
graph = GenGraph(rail_dict,directed=False)
```

or changing from a directed graph by the function `Dir2Undir`.

In [6]:
# whole directed graph
all_railway = {}
for rail in railway.values():
    all_railway.update(rail)

graph = GenGraph(all_railway)
nx.write_edgelist(graph,'graph/whole_directed_graph.g')

In [7]:
# whole undirected graph
all_railway = {}
for rail in railway.values():
    all_railway.update(rail)

graph = GenGraph(all_railway,directed=False)
nx.write_edgelist(graph,'graph/whole_undirected_graph.g')

In [8]:
# stratified graph

for t in railway_type:
    graph = GenGraph(railway[t])
    nx.write_edgelist(graph,'graph/'+t+'_directed_graph.g')
    graph = GenGraph(railway[t],directed=False)
    nx.write_edgelist(graph,'graph/'+t+'_undirected_graph.g')

You can save and read graph by:

```
nx.write_edgelist(graph,'railway.g')
graph = nx.read_edgelist('railway.g')
```